In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout, MultiHeadAttention, LayerNormalization
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, MultiHeadAttention, LayerNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, MultiHeadAttention, LayerNormalization
import os

D:\stock_price_prediction\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Generate synthetic data (e.g., exponential decay)
def simulate_ode(k=0.1, y0=1.0, t_max=10, num_points=1000):
    t = np.linspace(0, t_max, num_points)
    y = y0 * np.exp(-k * t)
    return t, y


In [4]:


def preprocess_stock_data(csv_path, date_column='Date', close_column='Close', test_size=0.2, val_size=0.1, time_steps=60):
    """
    Prepares stock market price data for time series modeling.
    
    Args:
        csv_path (str): Path to the CSV file containing the data.
        date_column (str): Name of the date column in the CSV.
        close_column (str): Name of the closing price column in the CSV.
        test_size (float): Proportion of the data for testing.
        val_size (float): Proportion of the training data for validation.
        time_steps (int): Number of past time steps to use for each sample.
    
    Returns:
        X_train, y_train: Training data and labels.
        X_val, y_val: Validation data and labels.
        X_test, y_test: Testing data and labels.
        scaler: Fitted MinMaxScaler instance for inverse scaling.
    """
    # Load the dataset
    data = pd.read_csv(csv_path, parse_dates=[date_column])
    data.sort_values(by=date_column, inplace=True)
    
    # Extract the 'close' column for scaling
    close_prices = data[close_column].values.reshape(-1, 1)
    
    # Scale the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_close = scaler.fit_transform(close_prices)
    
    # Create sequences of time_steps
    X, y = [], []
    for i in range(time_steps, len(scaled_close)):
        X.append(scaled_close[i-time_steps:i])
        y.append(scaled_close[i])
    
    X, y = np.array(X), np.array(y)
    
    # Split data into train, validation, and test sets
    train_size = int((1 - test_size) * len(X))
    val_size = int(val_size * train_size)
    
    X_train, X_temp = X[:train_size], X[train_size:]
    y_train, y_temp = y[:train_size], y[train_size:]
    
    X_val, X_test = X_temp[:val_size], X_temp[val_size:]
    y_val, y_test = y_temp[:val_size], y_temp[val_size:]
    
    return X_train, y_train, X_val, y_val, X_test, y_test, scaler



In [5]:

# Get the directory of the current script
current_dir = os.getcwd()
# Construct the path to the data folder (same level as the tuning folder)
data_folder = os.path.join(current_dir, "..", "inputs")
data_path_google  = os.path.join(data_folder, "google_stock_cleaned.csv")



X_train, y_train, X_val, y_val, X_test, y_test, scaler = preprocess_stock_data(
    data_path_google, date_column='Date', close_column='Close', time_steps=60
)

print(f"Train shape: {X_train.shape}, {y_train.shape}")
print(f"Validation shape: {X_val.shape}, {y_val.shape}")
print(f"Test shape: {X_test.shape}, {y_test.shape}")

Train shape: (554, 60, 1), (554, 1)
Validation shape: (55, 60, 1), (55, 1)
Test shape: (84, 60, 1), (84, 1)


In [21]:


def objective(trial, model_type):
    # Common hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])

    # Define the model based on the type
    model = Sequential()
    
    if model_type == 'rnn':
        num_layers = trial.suggest_int('num_layers', 1, 3)
        units = trial.suggest_int('units', 32, 256, step=32)
        for _ in range(num_layers):
            model.add(SimpleRNN(units, activation='tanh', return_sequences=True if _ < num_layers - 1 else False))
        model.add(Dense(1))

    elif model_type == 'lstm':
        num_layers = trial.suggest_int('num_layers', 1, 3)
        units = trial.suggest_int('units', 32, 256, step=32)
        for _ in range(num_layers):
            model.add(LSTM(units, activation='tanh', return_sequences=True if _ < num_layers - 1 else False))
        model.add(Dense(1))
        
    elif model_type == 'transformer':
        num_heads = trial.suggest_int('num_heads', 2, 8)
        key_dim = trial.suggest_int('key_dim', 16, 64, step=16)
        ff_units = trial.suggest_int('ff_units', 32, 128, step=32)
    
        # Input Layer
        input_layer = tf.keras.layers.Input(shape=(None, 1))
    
        # Transformer Encoder Layer
        attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(input_layer, input_layer)
        attention_output = LayerNormalization()(attention_output)
        attention_output = tf.keras.layers.Dense(ff_units, activation='relu')(attention_output)
    
         # Add a Dense layer for regression output
        output_layer = tf.keras.layers.Dense(1)(attention_output)
    
         # Create a Model instead of Sequential
        model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    elif model_type == 'neural_net':
        num_layers = trial.suggest_int('num_layers', 1, 3)
        units = trial.suggest_int('units', 32, 256, step=32)
        for _ in range(num_layers):
            model.add(Dense(units, activation='relu'))
        model.add(Dense(1))

    elif model_type == 'ode':
        num_layers = trial.suggest_int('num_layers', 1, 3)
        units = trial.suggest_int('units', 32, 256, step=32)
        activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])

       # Input layer to handle sequences
        input_layer = tf.keras.layers.Input(shape=(None, 1))
    
        # Stack Dense layers for feature extraction
        x = input_layer
        for _ in range(num_layers):
            x = tf.keras.layers.Dense(units, activation=activation)(x)
    
        # Approximate the derivative using a Dense layer
        derivative_layer = tf.keras.layers.Dense(units, activation=activation)(x)
    
        # Output layer for regression
        output_layer = tf.keras.layers.Dense(1)(derivative_layer)
    
        # Define the functional model
        model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    # Train the model
    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=10,
        batch_size=batch_size,
        verbose=0,
    )

    # Return the validation loss for Optuna to minimize
    val_loss = history.history['val_loss'][-1]
    return val_loss


In [7]:
study_rnn = optuna.create_study(direction='minimize')
study_rnn.optimize(lambda trial: objective(trial, model_type='rnn'), n_trials=100)
print("Best RNN parameters:", study_rnn.best_params)


[I 2024-12-09 23:51:04,401] A new study created in memory with name: no-name-eb3095a2-db20-4552-81cf-6692d4e8ec68
C:\Users\Kuzey\AppData\Local\Temp\ipykernel_8792\2965101208.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-12-09 23:51:10,540] Trial 0 finished with value: 0.07908274233341217 and parameters: {'learning_rate': 0.03883410995265465, 'batch_size': 128, 'num_layers': 2, 'units': 128}. Best is trial 0 with value: 0.07908274233341217.
[I 2024-12-09 23:51:18,682] Trial 1 finished with value: 0.3876410722732544 and parameters: {'learning_rate': 0.04715545756972078, 'batch_size': 16, 'num_layers': 1, 'units': 256}. Best is trial 0 with value: 0.07908274233341217.
[I 2024-12-09 23:51:26,152] Trial 2 finished with value: 0.005880710203200579 and

Best RNN parameters: {'learning_rate': 0.0007971184552975506, 'batch_size': 16, 'num_layers': 2, 'units': 256}


Best RNN parameters: {'learning_rate': 0.0007971184552975506, 'batch_size': 16, 'num_layers': 2, 'units': 256}
Best is trial 95 with value: 0.0006560706533491611.

Best RNN Architecture:


Model: "sequential_100"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_202 (SimpleRNN)           │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_203 (SimpleRNN)           │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_100 (Dense)                    │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
study_lstm = optuna.create_study(direction='minimize')
study_lstm.optimize(lambda trial: objective(trial, model_type='lstm'), n_trials=100)
print("Best LSTM parameters:", study_lstm.best_params)


[I 2024-12-10 00:14:23,787] A new study created in memory with name: no-name-c8d80d0e-af6c-401c-bbc2-1b59e81d884e
C:\Users\Kuzey\AppData\Local\Temp\ipykernel_8792\2965101208.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-12-10 00:14:45,341] Trial 0 finished with value: 0.009233498945832253 and parameters: {'learning_rate': 0.0006852436717290838, 'batch_size': 16, 'num_layers': 3, 'units': 32}. Best is trial 0 with value: 0.009233498945832253.
[I 2024-12-10 00:14:55,129] Trial 1 finished with value: 0.0018365848809480667 and parameters: {'learning_rate': 0.009147143248861594, 'batch_size': 64, 'num_layers': 2, 'units': 64}. Best is trial 1 with value: 0.0018365848809480667.
[I 2024-12-10 00:15:33,713] Trial 2 finished with value: 0.000876990321557

Best LSTM parameters: {'learning_rate': 0.006683068666642961, 'batch_size': 16, 'num_layers': 1, 'units': 64}


Best is trial 25 with value: 0.0006630570278503001. Best LSTM parameters: {'learning_rate': 0.006683068666642961, 'batch_size': 16, 'num_layers': 1, 'units': 64}


In [18]:
study_transformer = optuna.create_study(direction='minimize')
study_transformer.optimize(lambda trial: objective(trial, model_type='transformer'), n_trials=100)
print("Best Transformer parameters:", study_transformer.best_params)


[I 2024-12-10 01:05:14,543] A new study created in memory with name: no-name-f368775e-882b-4153-83d1-b4f6647484a3
C:\Users\Kuzey\AppData\Local\Temp\ipykernel_8792\1344150854.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-12-10 01:05:24,511] Trial 0 finished with value: 0.3884499967098236 and parameters: {'learning_rate': 0.0013044860677201011, 'batch_size': 32, 'num_heads': 6, 'key_dim': 64, 'ff_units': 128}. Best is trial 0 with value: 0.3884499967098236.
[I 2024-12-10 01:05:33,791] Trial 1 finished with value: 0.47836169600486755 and parameters: {'learning_rate': 0.00040169471354867215, 'batch_size': 16, 'num_heads': 7, 'key_dim': 16, 'ff_units': 64}. Best is trial 0 with value: 0.3884499967098236.
[I 2024-12-10 01:05:44,275] Trial 2 finished w

Best Transformer parameters: {'learning_rate': 0.042863058793002264, 'batch_size': 128, 'num_heads': 6, 'key_dim': 48, 'ff_units': 64}



Best is trial 28 with value: 0.206059530377388. Best Transformer parameters: {'learning_rate': 0.042863058793002264, 'batch_size': 128, 'num_heads': 6, 'key_dim': 48, 'ff_units': 64}

In [13]:
study_nn = optuna.create_study(direction='minimize')
study_nn.optimize(lambda trial: objective(trial, model_type='neural_net'), n_trials=100)
print("Best Neural Network parameters:", study_nn.best_params)


[I 2024-12-10 00:47:51,986] A new study created in memory with name: no-name-2d946ea2-eeeb-4b0c-88a5-d7b0aca4b5e9
C:\Users\Kuzey\AppData\Local\Temp\ipykernel_8792\2965101208.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-12-10 00:47:57,253] Trial 0 finished with value: 0.051648106426000595 and parameters: {'learning_rate': 0.00041853509325991513, 'batch_size': 64, 'num_layers': 3, 'units': 192}. Best is trial 0 with value: 0.051648106426000595.
[I 2024-12-10 00:47:59,855] Trial 1 finished with value: 0.0675826147198677 and parameters: {'learning_rate': 0.005830496138350314, 'batch_size': 128, 'num_layers': 1, 'units': 256}. Best is trial 0 with value: 0.051648106426000595.
[I 2024-12-10 00:48:02,517] Trial 2 finished with value: 0.052010793238878

Best Neural Network parameters: {'learning_rate': 0.01407170194768306, 'batch_size': 32, 'num_layers': 2, 'units': 128}


Best is trial 11 with value: 0.020302429795265198. Best Neural Network parameters: {'learning_rate': 0.01407170194768306, 'batch_size': 32, 'num_layers': 2, 'units': 128}


In [22]:
study_ode = optuna.create_study(direction='minimize')
study_ode.optimize(lambda trial: objective(trial, model_type='ode'), n_trials=100)
print("Best ODE parameters:", study_ode.best_params)


[I 2024-12-10 01:49:09,451] A new study created in memory with name: no-name-f63335c6-3834-4d31-8094-cbb773e09340
C:\Users\Kuzey\AppData\Local\Temp\ipykernel_8792\3026521219.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-12-10 01:49:13,009] Trial 0 finished with value: 0.05303420126438141 and parameters: {'learning_rate': 0.00015366902498731774, 'batch_size': 64, 'num_layers': 2, 'units': 64, 'activation': 'tanh'}. Best is trial 0 with value: 0.05303420126438141.
[I 2024-12-10 01:49:17,174] Trial 1 finished with value: 0.055973272770643234 and parameters: {'learning_rate': 0.008660362139318797, 'batch_size': 16, 'num_layers': 2, 'units': 64, 'activation': 'tanh'}. Best is trial 0 with value: 0.05303420126438141.
[I 2024-12-10 01:49:20,935] Trial 

Best ODE parameters: {'learning_rate': 0.018419286639189042, 'batch_size': 16, 'num_layers': 1, 'units': 224, 'activation': 'tanh'}


Best is trial 71 with value: 0.02775776945054531. Best ODE parameters: {'learning_rate': 0.018419286639189042, 'batch_size': 16, 'num_layers': 1, 'units': 224, 'activation': 'tanh'}

Here’s how to use this function to print the architecture for each model: